In [1]:
import numpy as np
local_data_dir = './Datasets/'

train_encodings = np.load(local_data_dir+"train_encodings.npy",allow_pickle=True)
train_labels = np.load(local_data_dir+"train_labels.npy",allow_pickle=True)
test_encodings = np.load(local_data_dir+"test_encodings.npy",allow_pickle=True)
test_labels = np.load(local_data_dir+"test_labels.npy",allow_pickle=True)

In [2]:
import tensorflow as tf
num_classes = 10
batch_size = 128

classifer_train_dict = {'train': train_encodings, 'labels': tf.one_hot(train_labels,num_classes)}
classifer_train_iterator = (
    tf.data.Dataset.from_tensor_slices(classifer_train_dict)
    .shuffle(10000)
    .repeat(-1) 
    .batch(batch_size)).make_one_shot_iterator()
classifer_train_batch = classifer_train_iterator.get_next()
def classifer_train_images(sess):
    return sess.run(classifer_train_batch)

In [3]:
from six.moves import xrange

def single_layer_classifier(h, l, nclass):
  logits = tf.layers.dense(h, nclass)
  output = tf.argmax(logits, 1)
  loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=l)
  return loss, output

sub_dim = 256
num_latents = 1
embedding_dim = sub_dim*num_latents

# assembly the single-layer classifer
x_classifer = tf.placeholder(tf.float32, shape=(None, embedding_dim))
y_classifer_label = tf.placeholder(tf.int32, shape = (None, num_classes))
loss, output = single_layer_classifier(x_classifer, y_classifer_label, num_classes)
cross_loss = tf.reduce_mean(loss)

# training
class_lr = 0.01
#lr = 0.01
#class_lr = tf.train.exponential_decay(lr, 0,
#                                        decay_steps=2000,
#                                        decay_rate=0.1)
optimizer = tf.train.AdamOptimizer(class_lr)
train_op = optimizer.minimize(cross_loss)
train_cross_error = []
num_classifer_updates = 30000

sess = tf.train.SingularMonitoredSession()
for i in xrange(num_classifer_updates):
  classifer_data = classifer_train_images(sess)
  feed_dict = {x_classifer: classifer_data['train'], y_classifer_label: classifer_data['labels']}
  if i!=num_classifer_updates:
    results = sess.run([train_op, cross_loss],
                       feed_dict=feed_dict)
    train_cross_error.append(results[1])
  else:
    results = sess.run([train_op, cross_error],
                       feed_dict=feed_dict)
    train_cross_error.append(results[1])
  if (i+1) % 100 == 0:
    print('%d iterations' % (i+1))
    print('cross_error: %.3f' % np.mean(train_cross_error[-100:]))
    print()

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
100 iterations
cross_error: 2.204

200 iterations
cross_error: 2.159

300 iterations
cross_error: 2.156

400 iterations
cross_error: 2.127

500 iterations
cross_error: 2.109

600 iterations
cross_error: 2.108

700 iterations
cross_error: 2.091

800 iterations
cross_error: 2.079

900 iterations
cross_error: 2.062

1000 iterations
cross_error: 2.056

1100 iterations
cross_error: 2.046

1200 iterations
cross_error: 2.030

1300 iterations
cross_error: 2.027

1400 iterations
cross_error: 2.018

1500 iterations
cross_error: 2.016

1600 iterations
cross_error: 1.994

1700 iterations
cross_error: 1.985

1800 iterations
cross_error: 1.987

1900 iterations
cross_error: 1.973

2000 iterations
cross_error: 1.972

2100 iterations
cross_error: 1.953

2200 iterations
cross_error: 1.958

2300 iterations
cross_error: 1.955

2400 iterations
cross_error: 1.932

2500 iterations
cross_err

In [4]:
# classifer accracy 
def classifer_accuracy(y_output, y_true):
  incorrect = np.array(y_output)-np.array(y_true)
  return 1-np.count_nonzero(incorrect)/len(y_output)

feed_dict = {x_classifer:test_encodings}
y_output = sess.run(output, feed_dict = feed_dict)
acc = classifer_accuracy(y_output, test_labels)
acc

0.5064